In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

class GeneticAlgorithm:
    def __init__(self, func, a, b, population_size=50, chromosome_length=20, 
                 crossover_rate=0.8, mutation_rate=0.01, generations=100):
        """
        遗传算法求解函数极大值
        
        参数:
        func: 目标函数
        a, b: 区间 [a, b]
        population_size: 种群大小
        chromosome_length: 染色体长度（二进制编码长度）
        crossover_rate: 交叉概率
        mutation_rate: 变异概率
        generations: 进化代数
        """
        self.func = func
        self.a = a
        self.b = b
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.generations = generations + 1
        
        # 初始化种群
        self.population = self.initialize_population()
        
    def initialize_population(self):
        """初始化种群 - 随机生成二进制染色体"""
        return [[random.randint(0, 1) for _ in range(self.chromosome_length)] 
                for _ in range(self.population_size)]
    
    def decode(self, chromosome):
        """将二进制染色体解码为实际变量值"""
        decimal_value = 0
        for i, gene in enumerate(chromosome):
            decimal_value += gene * (2 ** (self.chromosome_length - 1 - i))
        
        # 映射到区间 [a, b]
        return self.a + decimal_value * (self.b - self.a) / (2 ** self.chromosome_length - 1)
    
    def fitness(self, chromosome):
        """计算适应度 - 函数值"""
        x = self.decode(chromosome)
        return self.func(x)
    
    def selection(self, fitnesses):
        """选择操作 - 轮盘赌选择"""
        total_fitness = sum(fitnesses)
        # 避免除零错误
        if total_fitness == 0:
            probabilities = [1/len(fitnesses)] * len(fitnesses)
        else:
            probabilities = [f/total_fitness for f in fitnesses]
        
        # 累积概率
        cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
        
        # 轮盘赌选择
        selected = []
        for _ in range(self.population_size):
            r = random.random()
            for i, cp in enumerate(cumulative_probabilities):
                if r <= cp:
                    selected.append(self.population[i][:])  # 复制染色体
                    break
        return selected
    
    def crossover(self, parent1, parent2):
        """交叉操作 - 单点交叉"""
        if random.random() < self.crossover_rate:
            # 随机选择交叉点
            crossover_point = random.randint(1, self.chromosome_length - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            return child1, child2
        else:
            return parent1[:], parent2[:]  # 不交叉，直接复制
    
    def mutation(self, chromosome):
        """变异操作 - 位翻转变异"""
        for i in range(self.chromosome_length):
            if random.random() < self.mutation_rate:
                chromosome[i] = 1 - chromosome[i]  # 0变1，1变0
        return chromosome
    
    def evolve(self):
        """执行进化过程"""
        best_individual = None
        best_fitness = -float('inf')
        fitness_history = []
        
        for generation in range(self.generations):
            # 计算适应度
            fitnesses = [self.fitness(ind) for ind in self.population]
            
            # 更新最佳个体
            current_best_fitness = max(fitnesses)
            current_best_index = fitnesses.index(current_best_fitness)
            
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_individual = self.population[current_best_index][:]
            
            fitness_history.append(best_fitness)
            
            # 选择
            selected = self.selection(fitnesses)
            
            # 交叉
            new_population = []
            for i in range(0, self.population_size, 2):
                if i + 1 < self.population_size:
                    child1, child2 = self.crossover(selected[i], selected[i+1])
                    new_population.extend([child1, child2])
                else:
                    new_population.append(selected[i])
            
            # 变异
            self.population = [self.mutation(ind) for ind in new_population]
            
            # 精英保留：用上一代的最佳个体替换最差的个体
            if best_individual:
                worst_index = fitnesses.index(min(fitnesses))
                self.population[worst_index] = best_individual[:]
            
            # 打印进度
            if generation % 20 == 0:
                x_best = self.decode(best_individual)
                print(f"Generation {generation}: x = {x_best:.6f}, f(x) = {best_fitness:.6f}")
        
        return best_individual, best_fitness, fitness_history


In [66]:
# 1. 定义你的函数
def your_function(x):
    return x*np.log(x+10)-x**2  # 示例函数

# 2. 创建遗传算法实例
ga = GeneticAlgorithm(
    func=your_function,
    a=0,           # 区间起点
    b=2,          # 区间终点
    population_size=50,
    chromosome_length=20,
    crossover_rate=0.8,
    mutation_rate=0.01,
    generations=300
)

# 3. 执行进化
best_individual, best_fitness, history = ga.evolve()
best_x = ga.decode(best_individual)

Generation 0: x = 1.290473, f(x) = 1.462733
Generation 20: x = 1.267289, f(x) = 1.463231
Generation 40: x = 1.267181, f(x) = 1.463231
Generation 60: x = 1.267181, f(x) = 1.463231
Generation 80: x = 1.267181, f(x) = 1.463231
Generation 100: x = 1.267181, f(x) = 1.463231
Generation 120: x = 1.267181, f(x) = 1.463231
Generation 140: x = 1.267181, f(x) = 1.463231
Generation 160: x = 1.267181, f(x) = 1.463231
Generation 180: x = 1.267181, f(x) = 1.463231
Generation 200: x = 1.267181, f(x) = 1.463231
Generation 220: x = 1.267181, f(x) = 1.463231
Generation 240: x = 1.267181, f(x) = 1.463231
Generation 260: x = 1.267181, f(x) = 1.463231
Generation 280: x = 1.267181, f(x) = 1.463231
Generation 300: x = 1.267181, f(x) = 1.463231


In [54]:
def demo():
    # 定义测试函数 (在区间内有一个明显极大值)
    def test_function(x):
        return -x**2 + 4*x + 5  # 最大值在 x=2 处，f(2)=9
    
    # 参数设置
    a, b = 0, 5  # 搜索区间
    print(f"求解函数 f(x) = -x² + 4x + 5 在区间 [{a}, {b}] 上的极大值")
    print("理论最大值: f(2) = 9")
    print("-" * 50)
    
    # 创建遗传算法实例
    ga = GeneticAlgorithm(
        func=test_function,
        a=a,
        b=b,
        population_size=50,
        chromosome_length=20,
        crossover_rate=0.8,
        mutation_rate=0.01,
        generations=100
    )
    
    # 执行进化
    best_individual, best_fitness, fitness_history = ga.evolve()
    
    # 输出结果
    best_x = ga.decode(best_individual)
    print("\n" + "=" * 50)
    print("最终结果:")
    print(f"最优解 x = {best_x:.8f}")
    print(f"函数值 f(x) = {best_fitness:.8f}")
    print(f"理论最优 x = 2.000000, f(x) = 9.000000")
    print(f"误差: x误差 = {abs(best_x-2):.8f}, 函数值误差 = {abs(best_fitness-9):.8f}")
    
    # 绘制进化过程
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_history)
    plt.title('Genetic Algorithm Evolution Process')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.grid(True)
    plt.show()
    
    # 绘制函数图像和找到的最优点
    x_vals = np.linspace(a, b, 100)
    y_vals = test_function(x_vals)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x_vals, y_vals, 'b-', label='f(x) = -x² + 4x + 5')
    plt.plot(best_x, best_fitness, 'ro', markersize=10, label=f'Found Maximum (x={best_x:.4f})')
    plt.axvline(x=2, color='g', linestyle='--', alpha=0.7, label='Theoretical Maximum (x=2)')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('Function and Found Maximum')
    plt.legend()
    plt.grid(True)
    plt.show()

# 你可以用其他函数测试
def custom_demo():
    """使用自定义函数测试"""
    # 例如：f(x) = -x^4 + 3x^3 - 2x^2 + x，在[0, 2]区间
    def custom_func(x):
        return -x**4 + 3*x**3 - 2*x**2 + x
    
    ga = GeneticAlgorithm(
        func=custom_func,
        a=0,
        b=2,
        population_size=50,
        generations=100
    )
    
    best_individual, best_fitness, fitness_history = ga.evolve()
    best_x = ga.decode(best_individual)
    
    print(f"\n自定义函数结果:")
    print(f"最优解 x = {best_x:.8f}")
    print(f"函数值 f(x) = {best_fitness:.8f}")
    
    # 验证：用 numpy 求精确解对比
    x_vals = np.linspace(0, 2, 1000)
    y_vals = custom_func(x_vals)
    exact_max_x = x_vals[np.argmax(y_vals)]
    exact_max_y = np.max(y_vals)
    
    print(f"精确解 x = {exact_max_x:.8f}")
    print(f"精确值 f(x) = {exact_max_y:.8f}")
